In [25]:
import requests
def fetch_brain_viewer_details(biosample_id):
    bfidic = {
        "222": 100,
        "244": 116,
    }
    bfi_value = bfidic.get(str(biosample_id))
    if not bfi_value:
        return None
    url = f"http://dev2adi.humanbrain.in:8000/GW/getBrainViewerDetails/IIT/V1/SS-{bfi_value}:-1:-1"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error if the response code isn't 200
        return response.json()  # Or response.text based on the expected data format
    except requests.exceptions.RequestException as e:
        return None

In [26]:
def get_haematoxylin_and_eosin_metadata(biosample_id, section_number):
    # Fetch the JSON data
    json_data = fetch_brain_viewer_details(biosample_id)
    if not json_data:
        return None

    # Get HEOS section data safely
    heos_data = json_data.get('thumbNail', {}).get(
        "Haematoxylin and Eosin", [])
    if not heos_data:
        return None

    # Try exact match first
    exact_matches = [item for item in heos_data if str(
        item.get('position_index')) == str(section_number)]
    if exact_matches:
        return exact_matches

    # If not found, try nearest by numeric difference
    try:
        section_number_int = int(section_number)
    except ValueError:
        return None

    valid_indices = [item.get('position_index') for item in heos_data if isinstance(
        item.get('position_index'), int)]
    if not valid_indices:
        return None

    nearest_index = min(
        valid_indices, key=lambda x: abs(x - section_number_int))
    nearest_matches = [item for item in heos_data if item.get(
        'position_index') == nearest_index]

    return nearest_matches if nearest_matches else None

In [27]:
get_haematoxylin_and_eosin_metadata("222", "1")

[{'id': 195846,
  'position_index': 2,
  'jp2Path': '/data/storageIIT/humanbrain/analytics/222/HEOS/B_222_FB74-SL_1-ST_HEOS-SE_2_compressed.jp2',
  'pngPathLow': '/data/storageIIT/humanbrain/analytics/222/HEOS/B_222_FB74-SL_1-ST_HEOS-SE_2_thumbnail.jpg',
  'width': 22664,
  'height': 17694,
  'series': 309,
  'rigidrotation': 180,
  'notes': None,
  'trsdata': '{"center": [30521.670289637936, -16950.44869517583], "rotation": 4.208388660220902}',
  'export_status': 0,
  'rgb_val': None}]

In [28]:
def get_haematoxylin_and_eosin_metadata(biosample_id, section_number):
    # Fetch JSON response using predefined fetcher
    json_data = fetch_brain_viewer_details(biosample_id)
    if not json_data:
        return None

    heos_data = json_data.get('thumbNail', {}).get(
        'Haematoxylin and Eosin', [])
    if not heos_data:
        return None

    # Try exact match first
    exact_match = next(
        (item for item in heos_data if str(
            item.get('position_index')) == str(section_number)),
        None
    )

    match_item = exact_match

    # If exact match not found, find the nearest one by numeric distance
    if not match_item:
        try:
            section_number_int = int(section_number)
        except ValueError:
            return None

        valid_items = [item for item in heos_data if isinstance(
            item.get('position_index'), int)]
        if not valid_items:
            return None

        nearest_item = min(
            valid_items,
            key=lambda item: abs(item['position_index'] - section_number_int)
        )
        match_item = nearest_item

    # Extract only required fields
    filtered = {
        'height': match_item['height'],
        'width': match_item['width'],
        'rotation': match_item.get('rigidrotation', 0),
        'jp2_path_fragment': match_item['jp2Path'],
        'position_index': match_item['position_index']
    }

    return filtered

In [29]:
get_haematoxylin_and_eosin_metadata("222", "1")

{'height': 17694,
 'width': 22664,
 'rotation': 180,
 'jp2_path_fragment': '/data/storageIIT/humanbrain/analytics/222/HEOS/B_222_FB74-SL_1-ST_HEOS-SE_2_compressed.jp2',
 'position_index': 2}

git push -u origin master
git push -u github main

for f in *.jpg; do mv -- "$f" "${f%.jpg}.png"; done
